In [76]:
import igraph as ig
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import community as community_louvain
import matplotlib.cm as cm
import igraph as ig
import pandas as pd

In [77]:
# Carregando a matriz de adjacências e fazendo as devidas adaptações
matriz_adj = pd.read_csv('matriz_adj_final.csv', index_col='Aluno')
matriz_adj = matriz_adj.replace(',', '.', regex=True)
matriz_adj = matriz_adj.astype(float)

In [78]:
# Convertendo a matriz para um array numpy
adj_array = matriz_adj.values

# Criando o grafo a partir da matriz de adjacências
grafo = ig.Graph.Adjacency((adj_array > 0).tolist(), mode=ig.ADJ_UPPER)

In [79]:
# Adicionando os pesos às arestas
grafo.es['weight'] = adj_array[adj_array.nonzero()]

# Encontrar o peso máximo considerando os pesos invertidos
peso_max = max(grafo.es['weight'])

# Criar um subgrafo com arestas que têm 70% ou mais do peso máximo invertido
sub_g = grafo.subgraph_edges(grafo.es.select(weight_ge=peso_max*0.60), delete_vertices=False)

In [80]:
#Excutar o algoritmo de Louvain
comunidades = sub_g.community_multilevel(weights='weight', return_levels=False)

# Adicionando as comunidades ao grafo
sub_g.vs['comunidade'] = comunidades.membership

In [81]:
# Adicionando as cores de acordo com as comunidades
cores = cm.tab20b.colors
sub_g.vs['cor'] = [cores[com] for com in sub_g.vs['comunidade']]
sub_g.vs['size'] = 10

In [82]:
# Exibir informações sobre o subgrafo
num_comunidades = len(set(sub_g.vs['comunidade']))
print(f'Número de vértices no grafo: {len(sub_g.vs)}')
print(f'Número de arestas no grafo: {len(sub_g.es)}')
print(f'Número de comunidades detectadas: {num_comunidades}')

#Listar os vértices de cada comunidade
with open("comunidadesLouvain60Original.txt", "w") as f:
    for comunidade in range(num_comunidades):
        vertices = [v.index for v in sub_g.vs if v['comunidade'] == comunidade]
        f.write(f"Comunidade {comunidade}:\n")
        f.write(", ".join(map(str, vertices)) + "\n\n")

Número de vértices no grafo: 961
Número de arestas no grafo: 14465
Número de comunidades detectadas: 11


In [83]:
# Plotando o grafo
ig.plot(sub_g, 'Grafo_Louvain_60_Original.png', vertex_size=10, vertex_color=sub_g.vs['cor'], edge_width=0.5, edge_color='gray',mark_groups=True, bbox=(0, 0, 1000, 1000))